In [ ]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import Model, optimizers
from keras.layers import Dense, GlobalAveragePooling2D

# Dataset loading

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
val_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

train_path = './images/train/'
train_set = train_datagen.flow_from_directory(directory=train_path,target_size=(224,224),batch_size=32,color_mode="rgb",class_mode = 'sparse')
val_path = './images/val/'
validation_set = val_datagen.flow_from_directory(directory=val_path,target_size=(224,224),batch_size=32,color_mode="rgb",class_mode = 'sparse')
test_path = './images/test/'
test_set = test_datagen.flow_from_directory(directory=test_path,target_size=(224,224),batch_size=32,color_mode="rgb",class_mode = 'sparse')

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
class_names = np.unique(train_set.classes)
num_classes = len(class_names)
print('There are:', num_classes, 'classes')

# Model

In [ ]:
model = tf.keras.applications.resnet.ResNet101(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(img_height, img_width, 3),
    pooling=None,
    classes=num_classes,
    classifier_activation=None,
)

In [ ]:
x = model.output

x = GlobalAveragePooling2D()(x)
x = Dense(102, activation='softmax')(x) # sigmoid for binary classification

transfer_model = Model(inputs=model.input, outputs=x)

In [ ]:
# for i, layer in enumerate(transfer_model.layers):
#     print(i, layer.name, layer.trainable)

In [ ]:
# transfer_model.summary()

# Model training

In [ ]:
epochs=10

transfer_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizers.SGD(momentum=0.01, nesterov=True), metrics=["accuracy"])

In [ ]:
history = transfer_model.fit(train_set, batch_size = 32, epochs=epochs, validation_data=(validation_set))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Model evaluation

In [ ]:
# Generate generalization metrics
score = transfer_model.evaluate(test_set, verbose=1)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')